#  Track Analysis

In order to properly analyze our GPX tracks, we need to put all of them into the same _frame-of-reference_.   This is accomplished by the following algorithm...

* Create a set of variables with the initialized values
    * Current Coordinate = Starting Coordinate
        * $P_{\textrm{cur}} = P_{\textrm{start}}$
        * **NOTE:** Starting coordinate $P_{\textrm{start}}$ is defined below as a fixed variable
    * Distance Threshold = Small distance in meters
        * $d_{\textrm{thresh}} = 20 meters$
        * This is the distance between a possible point and the current point for consideration.
    * Step Distance = Small distance in meters.
        * $d_{\textrm{step}} = 20 meters$
        * This is the distance that the route will jump using the average angle
    * Waypoint list = empty array
        * $\hat{P}_{\textrm{waypoints}} = []$

## Step 0: Globals

In [1]:
database_path = 'bike_data.db'

start_coord = (39.5989743, -104.8609468)
end_coord   = (39.75428108249532, -105.00085402872664)

dist_thresh_m = 50
step_dist_m = 10

epsg_code = 32613

## Step 1: Import Required Libraries

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from ipyleaflet import Map, Marker, Polygon, Polyline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math, cmath
import re
from geographiclib.geodesic import Geodesic
from pyproj import CRS, Proj, Transformer

Setup the database.

In [3]:
# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT datasetId FROM point_list', conn)

Setup Proj

In [4]:
#  Setup the Projection Transformer
crs = CRS.from_epsg( epsg_code )
proj_dd2utm = Transformer.from_crs(crs.geodetic_crs, crs)
proj_utm2dd = Transformer.from_crs(crs, crs.geodetic_crs)
utm_zone = int(re.findall("\d+", crs.utm_zone)[0])
print('UTM Grid Zone: {}'.format(utm_zone))

(easting,northing) = proj_dd2utm.transform( start_coord[0], start_coord[1] )
start_coord_utm = np.array( [easting, northing], np.float64 )

UTM Grid Zone: 13


## Step 2: Import points 

In [5]:
#  For each dataset, load the points w.r.t. each dataset-id, sorted by time.
points_by_dataset = {}
for dataset_id in dataset_ids['datasetId']:
        
    #  Create a full track of the segment
    sql_query = 'SELECT * FROM point_list WHERE datasetId = {} ORDER BY timestamp'.format( dataset_id )
    points_by_dataset[dataset_id] = { 'points': pd.read_sql_query( sql_query, conn ) }

## Step 3: Helpful Utility Functions

We are doing a lot of trig here, so it's helpful to isolate some of the more standard methods to focus on the weirder stuff. 

In [6]:
def mean_angle(angles):
    return math.degrees(cmath.phase(sum(cmath.rect(1, math.radians(d)) for d in angles)/len(angles)))

def Cross( a, b ):
    return a[0] * b[1] - a[1] * b[0]
    
def Is_In_Front( anchor_point, test_point, angle ):
    
    #  Create a point to the side of the anchor to create a line we can compare
    side_point = ( anchor_point[0] + math.cos( math.radians( angle ) + math.pi/2.0 ),
                   anchor_point[1] + math.sin( math.radians( angle ) + math.pi/2.0 ) )
    
    p1 = ( test_point[0] - side_point[0],
           test_point[1] - side_point[1] )
    p2 = ( test_point[0] - anchor_point[0],
           test_point[1] - anchor_point[1] )
    res = Cross( p1, p2 )
    if res >= 0:
        return True
    return False   

## Step 4: Compute the starting point

The first point is weird cause it establishes the anchor points within our master route.

In [7]:
avg_pt_utm = np.array([0, 0], dtype=np.float64)
temp_angles = []
pt_count = 0

current_dataset_idx = {}
for dataset_id in dataset_ids['datasetId']:
    current_dataset_idx[dataset_id] = 0

#  Find all points that are within range of the starting point
for dataset_id in dataset_ids['datasetId']:
    point_dd  = [ points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                  points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] ]
    point_utm = np.array( [ points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                            points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] ], np.float64 )
    
    #  Check distance from starting point
    geod = Geodesic.WGS84.Inverse( start_coord[0], start_coord[1], point_dd[0], point_dd[1] )
    dist = np.linalg.norm( point_utm - start_coord_utm )
    
    if dist < dist_thresh_m:
        
            #  Compute angle to new point
            temp_angles.append( geod['azi1'])
            avg_pt_utm += point_utm
            pt_count += 1

#  Compute seed point
avg_angle = mean_angle( temp_angles )
start_pt = Geodesic.WGS84.Direct( start_coord[0], start_coord[1], avg_angle, step_dist_m )
start_pt_utm = proj_dd2utm.transform( start_pt['lat2'], start_pt['lon2'] )

waypoint_list = [ [ np.array( [ start_pt['lat2'], start_pt['lon2'] ] ), 
                    np.array( [ start_pt_utm[0], start_pt_utm[1] ], np.float64 ),
                    avg_angle ] ]
display( waypoint_list )

#  Update our iterators past all points "behind" the coordinate
for dataset_id in dataset_ids['datasetId']:

    while True:
        point_dd  = np.array( [ points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                                points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] ], np.float64 )
        point_utm = np.array( [ points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                                points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] ], np.float64 )
        
        # Update the indeces until the next coordinate is no longer in "front"
        if ( not Is_In_Front( waypoint_list[-1][0], point_dd, waypoint_list[-1][2] ) ) and ( np.linalg.norm( point_utm - waypoint_list[-1][1] ) < step_dist_m ):
            print('Current Dataset: {}, IDX: {}, Rejecting: {}'.format( dataset_id, current_dataset_idx[dataset_id], point_dd ))
            current_dataset_idx[dataset_id] += 1
        else:
            break
print(current_dataset_idx)    

[[array([  39.59893254, -104.86084364]),
  array([ 511947.54865579, 4383253.44548177]),
  117.62135375163585]]

Current Dataset: 1, IDX: 0, Rejecting: [  39.598945 -104.860885]
Current Dataset: 1, IDX: 1, Rejecting: [  39.598934 -104.86085 ]
{0: 0, 1: 2, 2: 0, 3: 0}


## Step 5:  Compute the remaining route

In [8]:
for i in range(0,600):
    print('Iteration: {}'.format(i))
    temp_angles = []
    
    #  Update Current Indices
    for dataset_id in dataset_ids['datasetId']:
        point_dd  = [ points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                      points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] ]
        point_utm = np.array( [ points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                                points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] ], np.float64 )
        
        #  Check distance from starting point
        geod = Geodesic.WGS84.Inverse( waypoint_list[-1][0][0], 
                                       waypoint_list[-1][0][1],
                                       point_dd[0],
                                       point_dd[1] )
        dist = np.linalg.norm( point_utm - waypoint_list[-1][1] )
    
        if dist < dist_thresh_m:
        
                #  Compute angle to new point
                temp_angles.append( geod['azi1'])

    #  Compute seed point
    avg_angle = mean_angle( temp_angles )
    next_point_dd = Geodesic.WGS84.Direct( waypoint_list[-1][0][0], 
                                           waypoint_list[-1][0][1],
                                           avg_angle, 
                                           step_dist_m )
    next_point_utm = proj_dd2utm.transform( next_point_dd['lat2'], next_point_dd['lon2'] )

    waypoint_list.append( [ np.array( [ next_point_dd['lat2'], next_point_dd['lon2'] ] ), 
                            np.array( [ next_point_utm[0], next_point_utm[1] ], np.float64 ),
                            avg_angle ] )
    
    #  Update our iterators past all points "behind" the coordinate
    for dataset_id in dataset_ids['datasetId']:

        while True:
            point_dd  = np.array( [ points_by_dataset[dataset_id]['points']['latitude'][current_dataset_idx[dataset_id]],
                                    points_by_dataset[dataset_id]['points']['longitude'][current_dataset_idx[dataset_id]] ], np.float64 )
            point_utm = np.array( [ points_by_dataset[dataset_id]['points']['easting'][current_dataset_idx[dataset_id]],
                                    points_by_dataset[dataset_id]['points']['northing'][current_dataset_idx[dataset_id]] ], np.float64 )
        
            # Update the indeces until the next coordinate is no longer in "front"
            if ( not Is_In_Front( waypoint_list[-1][0], point_dd, waypoint_list[-1][2] ) ) and ( np.linalg.norm( point_utm - waypoint_list[-1][1] ) < step_dist_m ):
                current_dataset_idx[dataset_id] += 1
            else:
                break  


Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38
Iteration: 39
Iteration: 40
Iteration: 41
Iteration: 42
Iteration: 43
Iteration: 44
Iteration: 45
Iteration: 46
Iteration: 47
Iteration: 48
Iteration: 49
Iteration: 50
Iteration: 51
Iteration: 52
Iteration: 53
Iteration: 54
Iteration: 55
Iteration: 56
Iteration: 57
Iteration: 58
Iteration: 59
Iteration: 60
Iteration: 61
Iteration: 62
Iteration: 63
Iteration: 64
Iteration: 65
Iteration: 66
Iteration: 67
Iteration: 68
Iteration: 69
Iteration: 70
Iteration: 71
It

## Visualize Results

In [9]:
centroid_pt = [0, 0]

#  Create the polyline and list of points
marker_list = [[start_coord[0],start_coord[1]]]
polyline   = []

for point in waypoint_list:
    marker_list.append( [point[0][0],point[0][1]] )
    centroid_pt[0] += point[0][0]
    centroid_pt[1] += point[0][1]
centroid_pt[0] /= len( waypoint_list )
centroid_pt[1] /= len( waypoint_list )
print( centroid_pt )

#  Build Map Visualization    
sector_map = Map( center=centroid_pt, zoom=14 )

#---------------------------------#
#-      Print the datasets       -#
#---------------------------------#
dataset_points = []
dataset_counter = 0.0
for dataset_id in dataset_ids['datasetId']:
    dataset_route = []
    for pidx in range( 0, len( points_by_dataset[dataset_id]['points']['latitude'] ) ):
        test_point = [ points_by_dataset[dataset_id]['points']['latitude'][pidx],
                       points_by_dataset[dataset_id]['points']['longitude'][pidx] ]
        #dataset_points.append( Marker( location=test_point, 
        #                               draggable=False,
        #                               color=matplotlib.colors.rgb2hex( plt.get_cmap('hsv')( dataset_counter / len(dataset_ids['datasetId']) ) ) ) )
        dataset_route.append( test_point )
    sector_map.add_layer( Polyline( locations= dataset_route,
                                    color=matplotlib.colors.rgb2hex( plt.get_cmap('hsv')( dataset_counter / len(dataset_ids['datasetId']) ) ),
                                    fill=False ) )
    
for p in dataset_points:
    sector_map.add_layer( p )
    
#-------------------------------------#
#-      Print the average route      -#
#-------------------------------------#
sector_map.add_layer( Marker( location=[start_coord[0],start_coord[1]] ) )
#for marker in marker_list:
#    marker = Marker( location=marker, draggable=False )
#    sector_map.add_layer( marker )
   
route_poly = Polyline( locations=marker_list,
                       color='blue',
                       fill=False )
sector_map.add_layer( route_poly )
    
#sector_map.layout.height="400px"
sector_map

[39.614482643981056, -104.8668086437336]


Map(center=[39.614482643981056, -104.8668086437336], controls=(ZoomControl(options=['position', 'zoom_in_text'…